In [58]:
import os
import shutil

In [6]:
import pysplit

In [83]:
from joblib import Parallel, delayed
import multiprocessing

In [1]:
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
readRDS = robjects.r['readRDS']

In [4]:
df = readRDS('/Users/jessssli/BurkeLab Dropbox/Data/hms_hysplit/hms_hysplit_initialization_distinct.rds')
df = pandas2ri.rpy2py_dataframe(df)

In [17]:
df[(df["state"] == "California") & (df["year"] == 2020)]

,lon,lat,height,year,month,day,hour,minute,ymd,state
5294356,-120.55,38.25,500.0,2020.0,1.0,1.0,14.0,0.0,2020-01-01,California
5294357,-120.55,38.25,1500.0,2020.0,1.0,1.0,14.0,0.0,2020-01-01,California
5294358,-120.55,38.25,2500.0,2020.0,1.0,1.0,14.0,0.0,2020-01-01,California
5294359,-120.55,38.25,500.0,2020.0,1.0,1.0,18.0,0.0,2020-01-01,California
5294360,-120.55,38.25,1500.0,2020.0,1.0,1.0,18.0,0.0,2020-01-01,California
...,...,...,...,...,...,...,...,...,...,...
5919902,-121.25,36.25,1500.0,2020.0,12.0,31.0,19.0,0.0,2020-12-31,California
5919903,-121.25,36.25,2500.0,2020.0,12.0,31.0,19.0,0.0,2020-12-31,California
5919907,-121.05,37.15,500.0,2020.0,12.0,31.0,20.0,0.0,2020-12-31,California
5919908,-121.05,37.15,1500.0,2020.0,12.0,31.0,20.0,0.0,2020-12-31,California


In [39]:
df.iloc[1:3,2:4]

,height,year
2,500.0,2011.0
3,500.0,2011.0


In [81]:
pysplit.generate_bulktraj(
    basename='',
    hysplit_working=r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test27/working/',
    output_dir=r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test28/out/',
    meteo_dir=r'/Users/jessssli/BurkeLab Dropbox/Data/meteorology/gdas1/',
    years=[2020],
    months=[1],
    hours=[14],
    altitudes=[500],
    coordinates=(38.25, -120.55),
    run = 72,
    meteoyr_2digits=True,
    outputyr_2digits=False,
    monthslice=slice(0, 1, 1),
    meteo_bookends=([4, 5], [1]),
    get_reverse=False,
    get_clipped=False,
    hysplit=r'/Users/jessssli/hysplit/exec/hyts_std'
)

In [ ]:
def generate_bulktraj_day():
    # Get points for the day
    
    for point in points:
        # Generate trajectory
        
        # Rename trajectory output file

In [ ]:
def rename_traj(coordinates, height, year, month, day, hour, out_dir, basename, mmm, seasons, dur, dxn):
    # Rename trajectory output file
    height_i_ = '0' + str(height_i) if height_i < 1000 else str(height_i)
    year_i = str(year_i)
    season_i = seasons[month_i]
    mmm_i = mmm[month_i]
    month_i = '0' + str(month_i) if month_i < 10 else str(month_i)
    day_i = '0' + str(day_i) if day_i < 10 else str(day_i)
    hour_i = '0' + str(hour_i) if hour_i < 10 else str(hour_i)
    lat_i = str(coordinates_i[0]).replace('.', 'p')
    lon_i = str(coordinates_i[1]).replace('.', 'p')
    fname_o = out_dir + basename + mmm_i + height_i_ + season_i + year_i + month_i + day_i + hour_i
    fname_n = f'{out_dir}traj-{basename}-{dxn}-{year_i[-2:]}-{month_i}-{day_i}-{hour_i}-1lat_{lat_i}_lon_{lon_i}-hgt_{height_i}-{dur}h'
    os.rename(fname_o, fname_n)
    

In [ ]:
def generate_bulktraj_point(basename, hysplit_working, output_dir, meteo_dir, years, months, hours, altitudes,
                           coordinates, run, meteoyr_2digits, outputyr_2digits, monthslice, meteo_bookends,
                           get_reverse, get_clipped, hysplit, mmm, seasons, out_dir, dur, dxn):
    pysplit.generate_bulktraj(
        basename = basename,
        hysplit_working = hysplit_working,
        output_dir = output_dir,
        meteo_dir = meteo_dir,
        years = years,
        months = months,
        hours = hours,
        altitudes = altitudes,
        coordinates = coordinates,
        run = run,
        meteoyr_2digits = meteoyr_2digits,
        outputyr_2digits = outputyr_2digits,
        monthslice = slice(0, 1, 1),
        meteo_bookends = meteo_bookends,
        get_reverse = get_reverse,
        get_clipped = get_clipped,
        hysplit = hysplit
    )
    
    rename_traj(coordinate, height = altitudes, year, month, day, hour, out_dir, basename, mmm, seasons, dur, dxn)

In [ ]:
def generate_bulktraj_point_(day):
    generate_bulktraj_point(day[0], day[1])

In [ ]:
def generate_bulktraj_parallel(
    num_cores,
    out_dir,
    basename,
    hysplit_working,
    exec_dir,
    meteo_dir,
    years,
    months,
    hours,
    altitudes,
    coordinates,
    run,
    meteoyr_2digits=True,
    outputyr_2digits=False,
    monthslice=slice(0, 32, 1),
    meteo_bookends=([4, 5], [1]),
    get_reverse=False,
    get_clipped=False,
    hysplit='C:\\hysplit4\\exec\\hyts_std'
):
    # Confirm meteorology directory exists
    assert os.path.isdir(meteo_dir)

    # Create working directory for each core
    if !os.path.isdir(exec_dir):
        os.mkdir(exec_dir)
    if exec_dir[-1] != '/':
        exec_dir = exec_dir + '/'
    exec_dir = [exec_dir + str(num_core) +
                '/' for num_core in range(1, num_cores)]
    for d in exec_dir:
        os.mkdir(d)

    # Confirm output directory exists or make it
    if !os.path.isdir(out_dir):
        os.mkdir(out_dir)
    if out_dir[-1] != '/':
        out_dir = out_dir = '/'

    seasons = {12: 'winter',
               1: 'winter',
               2: 'winter',
               3: 'spring',
               4: 'spring',
               5: 'spring',
               6: 'summer',
               7: 'summer',
               8: 'summer',
               9: 'autumn',
               10: 'autumn',
               11: 'autumn'}
    mmm = {1: 'jan',
           2: 'feb',
           3: 'mar',
           4: 'apr',
           5: 'may',
           6: 'jun',
           7: 'jul',
           8: 'aug',
           9: 'sep',
           10: 'oct',
           11: 'nov',
           12: 'dec'}
    dxn = 'fwd' if run > 0 else 'bwd'
    dur = abs(run)
    
    # Put args in list
    days = [
        coordinates
        altitudes
        years
        months
        hours
        monthslice
        run
        
    ]

    # Determine which working directory to use
    # Generate HYSPLIT trajectory

    Parallel(n_jobs = num_cores)(delayed(generate_bulktraj_point_)(day) for day in days)

    # Consolidate output to one folder
    for d in exec_dir:
        files = os.listdir(d)
        for f in files:
            shutil.move(d + f, out_dir)

In [ ]:
def generate_bulktraj_parallel(
    num_cores,
    out_dir,
    basename,
    hysplit_working,
    exec_dir,
    meteo_dir,
    years,
    months,
    hours,
    altitudes,
    coordinates,
    run,
    meteoyr_2digits=True,
    outputyr_2digits=False,
    monthslice=slice(0, 32, 1),
    meteo_bookends=([4, 5], [1]),
    get_reverse=False,
    get_clipped=False,
    hysplit='C:\\hysplit4\\exec\\hyts_std'
):
    # Confirm meteorology directory exists
    assert os.path.isdir(meteo_dir)

    # Create working directory for each core
    if !os.path.isdir(exec_dir):
        os.mkdir(exec_dir)
    if exec_dir[-1] != '/':
        exec_dir = exec_dir + '/'
    exec_dir = [exec_dir + str(num_core) +
                '/' for num_core in range(1, num_cores)]
    for d in exec_dir:
        os.mkdir(d)

    # Confirm output directory exists or make it
    if !os.path.isdir(out_dir):
        os.mkdir(out_dir)
    if out_dir[-1] != '/':
        out_dir = out_dir = '/'

    seasons = {12: 'winter',
               1: 'winter',
               2: 'winter',
               3: 'spring',
               4: 'spring',
               5: 'spring',
               6: 'summer',
               7: 'summer',
               8: 'summer',
               9: 'autumn',
               10: 'autumn',
               11: 'autumn'}
    mmm = {1: 'jan',
           2: 'feb',
           3: 'mar',
           4: 'apr',
           5: 'may',
           6: 'jun',
           7: 'jul',
           8: 'aug',
           9: 'sep',
           10: 'oct',
           11: 'nov',
           12: 'dec'}
    dxn = 'fwd' if run > 0 else 'bwd'
    dur = abs(run)

    # Determine which working directory to use
    # Generate HYSPLIT trajectory

    pysplit.generate_bulktraj(
        basename=basename,
        hysplit_working=hysplit_working,
        output_dir=output_dir,
        meteo_dir=meteo_dir,
        years=[2020],
        months=[1],
        hours=[14],
        altitudes=[500],
        coordinates=(38.25, -120.55),
        run=72,
        meteoyr_2digits=meteoyr_2digits,
        outputyr_2digits=outputyr_2digits,
        monthslice=slice(0, 1, 1),
        meteo_bookends=meteo_bookends,
        get_reverse=get_reverse,
        get_clipped=get_clipped,
        hysplit=hysplit
    )

    # Rename trajectory output file
    height_i_ = '0' + str(height_i) if height_i < 1000 else str(height_i)
    year_i = str(year_i)
    season_i = seasons[month_i]
    mmm_i = mmm[month_i]
    month_i = '0' + str(month_i) if month_i < 10 else str(month_i)
    day_i = '0' + str(day_i) if day_i < 10 else str(day_i)
    hour_i = '0' + str(hour_i) if hour_i < 10 else str(hour_i)
    lat_i = str(coordinates_i[0]).replace('.', 'p')
    lon_i = str(coordinates_i[1]).replace('.', 'p')
    fname_o = out_dir + basename + mmm_i + height_i_ + season_i + year_i + month_i + day_i + hour_i
    fname_n = f'{out_dir}traj-{basename}-{dxn}-{year_i[-2:]}-{month_i}-{day_i}-{hour_i}-1lat_{lat_i}_lon_{lon_i}-hgt_{height_i}-{dur}h'
    os.rename(fname_o, fname_n)
    
    Parallel(n_jobs = num_cores)(delayed(myfunc)(day) for day in days)

    # Consolidate output to one folder
    for d in exec_dir:
        files = os.listdir(d)
        for f in files:
            shutil.move(d + f, out_dir)

In [42]:
def myfunc(years):
    working_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/working/working_' + str(years)
    pysplit.generate_bulktraj(basename = 'colgate', 
                              hysplit_working = working_dir, 
                              output_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/trajectories/colgate2/', 
                              meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/met/gdas1/', 
                              years = [years], 
                              months = [1, 8], 
                              hours = [11, 17, 23], 
                              altitudes = [500, 1000, 1500], 
                              coordinates = (42.82, -75.54), 
                              run = -120, 
                              monthslice = slice(0, 32, 2), 
                              get_reverse = True, 
                              get_clipped = True,
                              hysplit = r'/Users/jessssli/hysplit/exec/hyts_std')


In [43]:
num_cores = multiprocessing.cpu_count()

In [44]:
years = [2007, 2011]

In [45]:
Parallel(n_jobs = num_cores)(delayed(myfunc)(yr) for yr in years)

[None, None]

In [82]:
__name__

'__main__'

In [86]:
def myfunc(x):
    print(x[0], "hello", x[1], x[2])

In [87]:
myfunc([1,2,3])

1 hello 2 3


In [88]:
xs = [[1,2,3],[4,5,6],[7,8,9]]

In [90]:
Parallel(n_jobs = 2)(delayed(myfunc)(x) for x in xs)

[None, None, None]

In [106]:
'a' + np.array(['b','c'])

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('<U1')) -> dtype('<U1')

In [110]:
[r'a/b/' + str(num_core) for num_core in range(1, 3)]

['a/b/1', 'a/b/2']

In [112]:
import shutil

In [116]:
shutil.move('/Users/jessssli/Downloads/my_dir3/gadm36_CAN_1_sp (1).rds', '/Users/jessssli/Downloads/my_dir2/')

'/Users/jessssli/Downloads/my_dir2/gadm36_CAN_1_sp (1).rds'

In [114]:
os.listdir('/Users/jessssli/Downloads/my_dir3/')

['gadm36_CAN_1_sp (1) copy.rds', 'gadm36_CAN_1_sp (1).rds']

In [118]:
src_dir = '/Users/jessssli/Downloads/my_dir3/'
files = os.listdir(src_dir)
for f in files:
    shutil.move(src_dir + f, '/Users/jessssli/Downloads/my_dir2/')

In [120]:
for i in [1,2,3]:
    print(i + 1)

2
3
4


In [122]:
'abc'[-2:]

'bc'

In [128]:
str((38.95, -125.34)[1])

'-125.34'

In [129]:
'ab.c'.replace('.','d')

'abdc'

In [130]:
abs(-2)

2

In [131]:
x = {1:'a',2:'b'}

In [132]:
x[1]

'a'

In [133]:
import time

In [146]:
xs = [1,7,2,3,4]

In [137]:
def myfunc(x):
    time.sleep(x)
    return(x)

In [147]:
Parallel(n_jobs = 3)(delayed(lambda x: myfunc(x))(x) for x in xs)

[1, 7, 2, 3, 4]

In [134]:
time.sleep(10)

In [138]:
myfunc(2)

2

In [150]:
import ray

In [151]:
ray.init()

2021-05-23 17:04:49,663	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.44',
 'raylet_ip_address': '192.168.1.44',
 'redis_address': '192.168.1.44:6379',
 'object_store_address': '/tmp/ray/session_2021-05-23_17-04-47_130672_93527/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-23_17-04-47_130672_93527/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-23_17-04-47_130672_93527',
 'metrics_export_port': 54988,
 'node_id': '2d75f1d495dad29a8a48abb3c5ecfea8675bbe6b945c1e32b20b7a1d'}

In [153]:
@ray.remote
def myfunc(x):
    time.sleep(x)
    return(x)

In [158]:
obj_ref = myfunc.remote(2)

In [159]:
ray.get(obj_ref)

2

In [162]:
@ray.remote
def slow_function():
    time.sleep(30)
    return 1

In [163]:
# Invocations of Ray remote functions happen in parallel.
# All computation is performed in the background, driven by Ray's internal event loop.
for _ in range(4):
    # This doesn't block.
    slow_function.remote()

In [ ]:
from mpi4py import MPI

In [ ]:
import sys
from mpi4py import MPI

size = MPI.COMM_WORLD.Get_size()
rank = MPI.COMM_WORLD.Get_rank()
name = MPI.Get_processor_name()

print("Hello, World! I am process ",rank," of ",size," on ",name)

In [ ]:
years = [2007, 2011]
months = [1, 8]
hours = [11, 17, 23]
altitudes = [500, 1000, 1500]
location = (42.82, -75.54)
runtime = -120

In [51]:
pysplit.generate_bulktraj(
    basename = 'colgate', 
    hysplit_working=r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test27/working/',
    output_dir=r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test27/out/',
    meteo_dir=r'/Users/jessssli/BurkeLab Dropbox/Data/meteorology/gdas1/',
    years = [2017, 2020],
    months = [1, 8], 
    hours = [11, 17, 23], 
    altitudes = [500, 1000, 1500], 
    coordinates = (42.82, -75.54), 
    run = -120,
    monthslice=slice(0, 32, 2), 
    meteo_bookends=([4, 5], [1]),
    get_reverse=False,
    get_clipped=False)

ValueError: invalid literal for int() with base 10: 'l'

In [ ]:
working_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/working/working_1/' #r'/Users/jessssli/hysplit/working/'
storage_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/trajectories/colgate3/'
meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/met/gdas1/'
hysplit = r'/Users/jessssli/hysplit/exec/hyts_std'

In [21]:
[i for i in range(10)][slice(1,10,2)]

[1, 3, 5, 7, 9]

In [34]:
[i for i in range(1,32)][slice(2, 3, 1)]

[3]

In [39]:
(lambda x: slice(x - 1, x, 1))(2)

slice(1, 2, 1)

In [75]:
working_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test27/working/'
storage_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test28/out/'
meteo_dir = r'/Users/jessssli/BurkeLab Dropbox/Data/meteorology/gdas1/'
hysplit = r'/Users/jessssli/hysplit/exec/hyts_std'

In [76]:
basename = 'pytraj'

In [77]:
years = [2020]
months = [1, 8]
hours = [11, 17, 23]
altitudes = [500, 1000, 1500]
location = (42.82, -75.54)
runtime = -120

In [78]:
pysplit.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
                          years, months, hours, altitudes, location, runtime,
                          monthslice=slice(0, 32, 2), get_reverse=True,
                          get_clipped=True, hysplit = hysplit)
os.system('say done!')

TypeError: 'module' object is not callable

# Parallelize PySPLIT  
Written by Jessica  
Last edited April 2021

Objective: given NOAA HMS HYSPLIT points, generate HYSPLIT forward trajectories efficiently.
1. Download and delete meteorology data via FTP periodically.
2. Create temporary working directories for each core running in parallel. Save trajectories to one output directory. Delete working directories when done generating trajectories.
3. Feed irregular parameters to HYSPLIT configuration.

In [4]:
import os

In [11]:
import pysplit
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
# ATTEMPT AT PARALLELIZING BASIC EXAMPLE ABOVE

In [ ]:
# for each year
# specify a unique working directory
# specify the same storage and meteo directories, basename, months, hours, alts, locs, and runtimes for all years
# and generate trajectories in bulk given the same params for all years and the different year and working_dir vals

In [12]:
def myfunc(years, 
#            working_dir = 0, 
           basename = 'colgate', 
           storage_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/trajectories/colgate2/',
           meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/met/gdas1/', 
           months = [1, 8], 
           hours = [11, 17, 23], 
           altitudes = [500, 1000, 1500], 
           location = (42.82, -75.54), 
           runtime = -120, 
           monthslice = slice(0, 32, 2), 
           get_reverse = True, 
           get_clipped = True, 
           hysplit = r'/Users/jessssli/hysplit/exec/hyts_std'):
    working_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/working/working_' + str(years)
    pysplit.generate_bulktraj(basename = basename, 
                              hysplit_working = working_dir, 
                              output_dir = storage_dir, 
                              meteo_dir = meteo_dir, 
                              years = [years], 
                              months = months, 
                              hours = hours, 
                              altitudes = altitudes, 
                              coordinates = location, 
                              run = runtime, 
                              monthslice = monthslice, 
                              get_reverse = get_reverse, 
                              get_clipped = get_clipped,
                              hysplit = hysplit)


In [1]:
import pysplit
from joblib import Parallel, delayed
import multiprocessing

In [42]:
def myfunc(years):
    working_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/working/working_' + str(years)
    pysplit.generate_bulktraj(basename = 'colgate', 
                              hysplit_working = working_dir, 
                              output_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/trajectories/colgate2/', 
                              meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/met/gdas1/', 
                              years = [years], 
                              months = [1, 8], 
                              hours = [11, 17, 23], 
                              altitudes = [500, 1000, 1500], 
                              coordinates = (42.82, -75.54), 
                              run = -120, 
                              monthslice = slice(0, 32, 2), 
                              get_reverse = True, 
                              get_clipped = True,
                              hysplit = r'/Users/jessssli/hysplit/exec/hyts_std')


In [43]:
num_cores = multiprocessing.cpu_count()

In [44]:
years = [2007, 2011]

In [45]:
Parallel(n_jobs = num_cores)(delayed(myfunc)(yr) for yr in years)

[None, None]

In [10]:
os.system('say done')

0

# Feed irregular initialization values

In [ ]:
pysplit.generate_bulktraj(basename = 'change_later',
                          hysplit_working = r'/Users/jessssli/Documents/GitHub/wildfire_home_leakage/work/get_HYSPLIT_height/new_working/',
                          output_dir = r'/Users/jessssli/Documents/GitHub/wildfire_home_leakage/work/get_HYSPLIT_height/new_output/',
                          meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/met/gdas1/',
                          years = 2007,
                          months = 8,
                          hours = 3,
                          altitudes = 1000,
                          coordinates = (42.82, -75.54),
                          run = 72,
                          meteoyr_2digits = True,
                          outputyr_2digits = False,
                          monthslice=slice(0, 32, 1),
                          meteo_bookends=([4, 5], [1]),
                          get_reverse = False,
                          get_clipped = False,
                          hysplit='C:\\hysplit4\\exec\\hyts_std',
                         )

In [15]:
slice(0, 32, 1)+1

TypeError: unsupported operand type(s) for +: 'slice' and 'int'

In [ ]:
pysplit.trajectory_generator

In [20]:
working_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/working/working_1/' #r'/Users/jessssli/hysplit/working/'
storage_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/trajectories/colgate3/'
meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/met/gdas1/'
hysplit = r'/Users/jessssli/hysplit/exec/hyts_std'

In [21]:
basename = 'colgate'

In [22]:
years = [2007, 2011]
months = [1, 8]
hours = [11, 17, 23]
altitudes = [500, 1000, 1500]
location = (42.82, -75.54)
runtime = -120

In [24]:
pysplit.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir, 
                          years, months, hours, altitudes, location, runtime, 
                          monthslice = slice(0, 32, 2), get_reverse = True, get_clipped = True,
                          hysplit = hysplit)

In [13]:
os.getcwd() + '/myfolder'

'/Users/jessssli/Documents/GitHub/wildfire_home_leakage/work/get_HYSPLIT_height/myfolder'

In [17]:
import numpy as np

In [20]:
np.array([i for i in range(100)][slice(0, 1, 1)]) + 1

array([1])

In [14]:
import numpy as np
import pandas

In [4]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [6]:
readRDS = robjects.r['readRDS']

In [9]:
df = readRDS('/Users/jessssli/BurkeLab Dropbox/Data/hms_hysplit/dat_hms_hysplit.rds')
df = pandas2ri.rpy2py_dataframe(df)

In [31]:
df

,lon,lat,height,year,month,day,hour,minute,ymd,duration
1,-81.130,22.839,500.0,2011.0,1.0,1.0,20.0,0.0,2011-01-01,200
2,-80.963,22.780,500.0,2011.0,1.0,1.0,21.0,0.0,2011-01-01,200
3,-82.871,22.847,500.0,2011.0,1.0,1.0,19.0,0.0,2011-01-01,400
4,-83.089,22.646,500.0,2011.0,1.0,1.0,19.0,0.0,2011-01-01,400
5,-79.212,21.576,500.0,2011.0,1.0,1.0,20.0,0.0,2011-01-01,200
...,...,...,...,...,...,...,...,...,...,...
501462,-111.716,35.468,500.0,2018.0,7.0,31.0,19.0,0.0,2018-07-31,2400
501463,-111.753,35.498,500.0,2018.0,7.0,31.0,19.0,0.0,2018-07-31,2400
501464,-111.984,36.168,500.0,2018.0,7.0,31.0,19.0,0.0,2018-07-31,2400
501465,-96.024,28.711,500.0,2019.0,12.0,2.0,19.0,0.0,2019-12-02,2400


In [39]:
df.iloc[1:3,2:4]

,height,year
2,500.0,2011.0
3,500.0,2011.0


In [11]:
import pysplit

In [ ]:
pysplit.generate_bulktraj(
    basename = 'pytraj',
    hysplit_working = r'/Users/jessssli/hysplit/exec/hyts_std/',
    output_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test13/',
    meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/test10/met/',
    years = ,
    months,
    hours,
    altitudes,
    coordinates,
    run,
    meteoyr_2digits=True,
    outputyr_2digits=False,
    monthslice=slice(0, 32, 1),
    meteo_bookends=([4, 5], [1]),
    get_reverse=False,
    get_clipped=False,
    hysplit='C:\\hysplit4\\exec\\hyts_std',
)

In [ ]:
working_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/working/working_1/' #r'/Users/jessssli/hysplit/working/'
storage_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/trajectories/colgate3/'
meteo_dir = r'/Users/jessssli/Documents/04 Sub-Annual Model of Wildfire PM/HYSPLIT Speed Test/met/gdas1/'
hysplit = r'/Users/jessssli/hysplit/exec/hyts_std'